In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itbr"
fs_method, fs_ratio = "rfcls", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9098 | AUPR: 0.6389 | Acc: 95.30%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9146 | AUPR: 0.6589 | Acc: 95.44%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9198 | AUPR: 0.6750 | Acc: 95.53%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9117 | AUPR: 0.6460 | Acc: 95.27%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9165 | AUPR: 0.6637 | Acc: 95.44%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9202 | AUPR: 0.6753 | Acc: 95.53%
best model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9202 | AUPR: 0.6753 | Acc: 95.53%
Internal test: AUC: 0.9274 | AUPR: 0.7144 | Acc: 95.30%
External test: AUC: 0.9180 | AUPR: 0.6733 | Acc: 95.20%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9106 | AUPR: 0.6139 | Acc: 94.86%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9131 | AUPR: 0.6291 | Acc: 95.08%
model(kernel=linear, C=1)
Valid:         AUC: 0.9129 | AUPR: 0.6311 | Acc: 95.13%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9131 | AUPR: 0.6291 | Acc: 95.08%
Internal test: AUC: 0.9167 | AUPR: 0.6597 | Acc: 94.89%
External test: AUC: 0.9110 | AUPR: 0.6051 | Acc: 94.44%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7718 | AUPR: 0.4307 | Acc: 94.51%
model(n_neighbors=5)
Valid:         AUC: 0.7990 | AUPR: 0.4719 | Acc: 94.93%
model(n_neighbors=7)
Valid:         AUC: 0.8181 | AUPR: 0.4989 | Acc: 95.02%
best model(n_neighbors=7)
Valid:         AUC: 0.8181 | AUPR: 0.4989 | Acc: 95.02%
Internal test: AUC: 0.8218 | AUPR: 0.5308 | Acc: 94.91%
External test: AUC: 0.8112 | AUPR: 0.5047 | Acc: 94.85%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8890 | AUPR: 0.4781 | Acc: 88.05%
best model()
Valid:         AUC: 0.8890 | AUPR: 0.4781 | Acc: 88.05%
Internal test: AUC: 0.8998 | AUPR: 0.5460 | Acc: 88.56%
External test: AUC: 0.8870 | AUPR: 0.4844 | Acc: 87.20%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9128 | AUPR: 0.6499 | Acc: 95.38%
model(C=10.0)
Valid:         AUC: 0.9131 | AUPR: 0.6506 | Acc: 95.29%
model(C=100.0)
Valid:         AUC: 0.9130 | AUPR: 0.6504 | Acc: 95.29%
best model(C=10.0)
Valid:         AUC: 0.9131 | AUPR: 0.6506 | Acc: 95.29%
Internal test: AUC: 0.9203 | AUPR: 0.6942 | Acc: 95.28%
External test: AUC: 0.9095 | AUPR: 0.6348 | Acc: 94.92%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8747 | AUPR: 0.5572 | Acc: 85.24%
model(criterion=log_loss)
Valid:         AUC: 0.8786 | AUPR: 0.5649 | Acc: 84.72%
model(criterion=entropy)
Valid:         AUC: 0.8786 | AUPR: 0.5649 | Acc: 84.72%
best model(criterion=log_loss)
Valid:         AUC: 0.8786 | AUPR: 0.5649 | Acc: 84.72%
Internal test: AUC: 0.8877 | AUPR: 0.6197 | Acc: 84.31%
External test: AUC: 0.8703 | AUPR: 0.5701 | Acc: 84.13%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9192 | AUPR: 0.6560 | Acc: 89.41%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9192 | AUPR: 0.6575 | Acc: 89.61%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9193 | AUPR: 0.6592 | Acc: 89.63%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9182 | AUPR: 0.6607 | Acc: 89.14%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9185 | AUPR: 0.6617 | Acc: 89.49%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9188 | AUPR: 0.6631 | Acc: 89.63%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9182 | AUPR: 0.6607 | Acc: 89.14%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9185 | AUPR: 0.6617 | Acc: 89.49%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9188 | AUPR: 0.6631 | Acc: 89.63%
best model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9193 | AUPR: 0.6592 | Acc: 8

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9197 | AUPR: 0.6465 | Acc: 95.13%
model(criterion=squared_error)
Valid:         AUC: 0.9197 | AUPR: 0.6464 | Acc: 95.12%
best model(criterion=friedman_mse)
Valid:         AUC: 0.9197 | AUPR: 0.6465 | Acc: 95.13%
Internal test: AUC: 0.9279 | AUPR: 0.7044 | Acc: 95.28%
External test: AUC: 0.9177 | AUPR: 0.6592 | Acc: 95.13%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9134 | AUPR: 0.6391 | Acc: 95.24%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9130 | AUPR: 0.6408 | Acc: 95.22%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9143 | AUPR: 0.6428 | Acc: 95.27%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9143 | AUPR: 0.6428 | Acc: 95.27%
Internal test: AUC: 0.9196 | AUPR: 0.6868 | Acc: 95.30%
External test: AUC: 0.9087 | AUPR: 0.6327 | Acc: 94.99%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)